In [1]:
import pandas as pd
import os
import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


In [3]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14572005430028988561
 xla_global_id: -1]

In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [8]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [46]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0


In [ ]:
sns.heatmap(train.corr(), annot = True)

In [ ]:
sns.pairplot(train)

# 파이캐럿 (전처리 X)

In [ ]:
pycaret_data = pd.get_dummies(data = train, columns = ['Gender'], prefix = 'Gender')

In [ ]:
from pycaret.classification import *
from pycaret.regression import *

In [ ]:
setup_clf = setup(data = pycaret_data, target = 'Target', fold_shuffle=True, use_gpu=True, silent=True)

In [ ]:
top10 = compare_models(sort='MAE', n_select=10, fold = 5)

In [ ]:
tuned_top10 = [tune_model(i, fold = 5) for i in top10]

In [ ]:
blender_top10 = blend_models(estimator_list=tuned_top10)

In [ ]:
final_model = finalize_model(blender_top10)
prediction = predict_model(final_model, data= test)

In [ ]:
prediction

In [ ]:
prediction['Label']

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)

# 딥러닝 기반

In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [43]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=10, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [44]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=45, mode='min')

In [ ]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [21]:
Y_prediction = model.predict(test).flatten()

In [22]:
Y_prediction

array([ 8.929717, 14.675825,  5.293755, ...,  9.04725 ,  8.42572 ,
       12.411996], dtype=float32)

In [39]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission.csv',index=False)

# Kfold

In [27]:
from sklearn.model_selection import StratifiedKFold

In [28]:
skf = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

In [29]:
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')
rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [30]:
nn_val_mae = []
nn_pred = np.array([0.0]*2924)
for i, (tr_idx, val_idx) in enumerate(skf.split(train_X, train_y)) :
    print(f'{i + 1} Fold Training.....')
    tr_x, tr_y = train_X.iloc[tr_idx], train_y[tr_idx]
    val_x, val_y = train_X.iloc[val_idx], train_y[val_idx]
    
    ### NN 모델
    model = Sequential()
    model.add(Dense(16, input_dim=len(train_X.columns), activation='elu'))
    model.add(Dense(32, activation='elu'))    
    model.add(Dense(64, activation='elu'))            
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(16, activation='elu'))
    model.add(Dense(1))

    ### ModelCheckPoint Fold마다 갱신
    cp = ModelCheckpoint(f'model_{i + 1}.h5', save_best_only = True, monitor = 'val_mae', mode = 'min', verbose = 0)
    ### 모델 compile
    model.compile(loss='mean_absolute_error',
                  optimizer='Nadam', 
                  metrics=['mae'])

    result = model.fit(tr_x, tr_y, validation_data = (val_x, val_y), epochs = 1000, batch_size = 32, callbacks = [es, cp, rlrp], verbose = 0)

    ### 최고 성능 기록 모델 Load
    best = load_model(f'model_{i + 1}.h5')
    ### validation predict
    val_pred = best.predict(val_x)
    ### Fold별 val_mae 산출
    fold_val_mae = result.history['val_loss'][np.argmin(result.history['val_loss'])]
    nn_val_mae.append(fold_val_mae)
    print(f'{i + 1} Fold val mae = {fold_val_mae}\n')

    ### Fold별 test 데이터에 대한 예측값 생성 및 앙상블
    fold_pred = best.predict(test).flatten()
    nn_pred += fold_pred

1 Fold Training.....
1 Fold val mae = 1.4200752973556519

2 Fold Training.....
2 Fold val mae = 1.525030493736267

3 Fold Training.....
3 Fold val mae = 1.4293029308319092

4 Fold Training.....
4 Fold val mae = 1.5648192167282104

5 Fold Training.....
5 Fold val mae = 1.4164764881134033

6 Fold Training.....
6 Fold val mae = 1.3439258337020874

7 Fold Training.....
7 Fold val mae = 1.3681161403656006

8 Fold Training.....
8 Fold val mae = 1.3389068841934204

9 Fold Training.....
9 Fold val mae = 1.4959264993667603

10 Fold Training.....
10 Fold val mae = 1.3552550077438354



In [31]:
np.mean(nn_val_mae)

1.4257834792137145

In [21]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(nn_pred / skf.n_splits)

submission

submission.to_csv('submission_fold.csv',index=False)

# feature 추가

In [214]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [215]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [216]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0


In [217]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])

In [218]:
foreign_body

0       0.1205
1       0.0185
2       0.1220
3       0.0590
4       0.0080
         ...  
1248    0.0000
1249    0.0200
1250    0.0155
1251    0.0040
1252    0.1965
Length: 1253, dtype: float64

In [220]:
foreign_body[47]

-0.06749999999999998

In [221]:
train_X['foreign body'] = foreign_body

In [222]:
#train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [223]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
#test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [224]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [225]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=11, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [226]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [ ]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [207]:
Y_prediction = model.predict(test).flatten()

In [208]:
Y_prediction

array([10.270859 , 14.649908 ,  5.4945755, ...,  9.137286 ,  8.677276 ,
       11.729927 ], dtype=float32)

In [209]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission.csv',index=False)

## 피쳐 추가 2

In [151]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [152]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [153]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0


In [154]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])
train_X['foreign body'] = foreign_body
train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [155]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [156]:
# 껍질의 넓이 ( a * b * π)
area = 0.5 * train_X['Lenght'] * 0.5 * train_X['Diameter'] * np.pi
train_X['Area'] = area

In [157]:
# 껍질의 넓이 
area = 0.5 * test['Lenght'] * 0.5 * test['Diameter'] * np.pi
test['Area'] = area

In [158]:
# 껍질의 둘레 ( 2π*(0.5 * √(a^2 + b^2)))
perimeter = np.pi * np.sqrt(0.5 * ((train_X['Lenght'] ** 2) + (train_X['Diameter'] ** 2)))
train_X['Perimeter'] = perimeter

In [159]:
# 껍질의 둘레
perimeter = np.pi * np.sqrt(0.5 * ((test['Lenght'] ** 2) + (test['Diameter'] ** 2)))
test['Perimeter'] = perimeter

In [162]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M,foreign body,Area,Perimeter,Volume
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1,0.1205,0.223328,1.701869,0.017122
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0,0.0185,0.106382,1.184103,0.006738
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0,0.1220,0.223210,1.686687,0.029017
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1,0.0590,0.170176,1.490602,0.019854
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0,0.0080,0.057216,0.864152,0.003433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0,0.0000,0.021638,0.530943,0.000577
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0,0.0200,0.096172,1.115431,0.005450
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0,0.0155,0.169057,1.479762,0.012961
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0,0.0040,0.117083,1.237345,0.008586


In [163]:
test

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M,foreign body,Area,Perimeter,Volume
0,0.595,0.470,0.155,1.1210,0.4515,0.1780,0.1550,1,0,0,0.3365,0.219637,1.684382,0.022696
1,0.580,0.450,0.150,0.9270,0.2760,0.1815,0.3600,0,0,1,0.1095,0.204989,1.630756,0.020499
2,0.260,0.205,0.070,0.0970,0.0415,0.0190,0.0305,0,1,0,0.0060,0.041862,0.735512,0.001954
3,0.590,0.460,0.130,1.1020,0.4550,0.2055,0.3300,0,0,1,0.1115,0.213157,1.661929,0.018474
4,0.595,0.465,0.140,1.1130,0.5175,0.2440,0.3050,1,0,0,0.0465,0.217300,1.677519,0.020281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,0.170,0.105,0.035,0.0340,0.0120,0.0085,0.0050,0,1,0,0.0085,0.014019,0.443872,0.000327
2920,0.435,0.345,0.115,0.4180,0.2220,0.0735,0.1060,0,1,0,0.0165,0.117869,1.233350,0.009037
2921,0.570,0.450,0.135,0.7940,0.3815,0.1415,0.2450,0,1,0,0.0260,0.201455,1.613262,0.018131
2922,0.460,0.350,0.120,0.4885,0.1930,0.1050,0.1550,0,1,0,0.0355,0.126449,1.284024,0.010116


In [ ]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=len(train_X.columns), activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

# 모델 학습
history = model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [182]:
history.history['val_loss'][np.argmin(history.history['val_loss'])]

1.5259870290756226

In [183]:
Y_prediction = model.predict(test).flatten()

In [184]:
Y_prediction

array([10.361692 , 14.353676 ,  5.2455287, ...,  9.15653  ,  8.703988 ,
       11.8237505], dtype=float32)

In [168]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission_1.5161_넓이둘레부피추가_실험.csv',index=False)

# 실험실

## 표준화

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
# StandardScaler객체 생성
scaler = StandardScaler()

# StandardScaler 로 데이터 셋 변환 .fit( ) 과 .transform( ) 호출
scaler.fit(train_X)
train_X_scaled = scaler.transform(train_X)

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
train_X_scaled = pd.DataFrame(data=train_X_scaled, columns=train_X.columns)

In [6]:
train_X_scaled

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.683382,0.632069,-0.629084,0.565272,0.150018,0.983043,0.490632,-0.659758,-0.689822,1.292920
1,-0.772731,-0.922116,-1.141679,-0.905870,-0.818704,-0.902715,-0.951302,-0.659758,1.449650,-0.773443
2,0.475366,0.832609,1.421295,0.970036,0.764655,0.652365,1.192304,-0.659758,1.449650,-0.773443
3,0.100937,-0.019686,0.908700,0.878090,0.842598,1.300315,0.680084,-0.659758,-0.689822,1.292920
4,-1.771209,-1.724275,-1.269828,-1.407577,-1.384349,-1.340640,-1.403880,-0.659758,1.449650,-0.773443
...,...,...,...,...,...,...,...,...,...,...
1248,-2.769687,-2.626705,-2.551316,-1.585474,-1.524646,-1.559602,-1.579298,-0.659758,1.449650,-0.773443
1249,-1.063954,-0.972251,-1.397977,-1.027799,-0.916690,-1.166364,-1.028486,-0.659758,1.449650,-0.773443
1250,0.017730,0.030449,-0.629084,-0.113332,0.254684,-0.160924,-0.421540,1.515707,-0.689822,-0.773443
1251,-0.647921,-0.721576,-0.757233,-0.790937,-0.696222,-0.639066,-0.846052,1.515707,-0.689822,-0.773443


# 딥러닝 기반

In [23]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from sklearn.model_selection import train_test_split

In [261]:
# train, test 데이터 분리
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=123)

In [262]:
X_train

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
743,0.680,0.540,0.155,1.5340,0.6710,0.3790,0.3840,0,0,1
955,0.285,0.215,0.060,0.0935,0.0310,0.0230,0.0300,0,1,0
920,0.575,0.445,0.135,0.8830,0.3810,0.2035,0.2600,1,0,0
238,0.510,0.400,0.145,0.5775,0.2310,0.1430,0.1770,0,1,0
40,0.695,0.560,0.190,1.4940,0.5880,0.3425,0.4850,0,0,1
...,...,...,...,...,...,...,...,...,...,...
104,0.400,0.315,0.100,0.3225,0.1430,0.0735,0.0910,0,1,0
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
114,0.585,0.460,0.165,1.0580,0.4860,0.2500,0.2940,1,0,0
1129,0.530,0.405,0.120,0.6320,0.2715,0.1480,0.1875,0,1,0


In [34]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=10, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [263]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=45, mode='min')

In [ ]:
# 모델 학습
history = model.fit(X_train, Y_train, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [265]:
# 예측 값과 실제 값의 비교
Y_prediction = model.predict(X_test).flatten()
# flatten : 데이터 배열이 몇 차원이든 모두 1차원으로 바꿔 읽기 쉽게 해 주는 함수

# 10개 실제값과 예측값 비교
for i in range(20):
    label = Y_test.values[i]
    prediction = Y_prediction[i]
    print("실제나이: {:.3f}, 예상나이: {:.3f}".format(label, prediction))

실제나이: 12.000, 예상나이: 8.457
실제나이: 8.000, 예상나이: 9.202
실제나이: 10.000, 예상나이: 10.826
실제나이: 12.000, 예상나이: 11.440
실제나이: 9.000, 예상나이: 9.474
실제나이: 10.000, 예상나이: 9.944
실제나이: 11.000, 예상나이: 8.314
실제나이: 13.000, 예상나이: 10.936
실제나이: 9.000, 예상나이: 8.988
실제나이: 10.000, 예상나이: 10.586
실제나이: 12.000, 예상나이: 9.630
실제나이: 8.000, 예상나이: 8.728
실제나이: 8.000, 예상나이: 9.122
실제나이: 13.000, 예상나이: 11.041
실제나이: 14.000, 예상나이: 9.606
실제나이: 8.000, 예상나이: 8.748
실제나이: 7.000, 예상나이: 8.558
실제나이: 6.000, 예상나이: 7.683
실제나이: 11.000, 예상나이: 10.699
실제나이: 11.000, 예상나이: 9.612


## 표준화 실험

In [43]:
from sklearn.preprocessing import StandardScaler

In [87]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [ ]:
train_X

In [88]:
train_X.drop('Gender', axis=1 , inplace = True)

# StandardScaler객체 생성
scaler = StandardScaler()

# StandardScaler 로 데이터 셋 변환 .fit( ) 과 .transform( ) 호출
scaler.fit(train_X)
train_X_scaled = scaler.transform(train_X)

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
train_X_scaled = pd.DataFrame(data=train_X_scaled, columns=train_X.columns)

train_X_scaled['Gender'] = train['Gender']

train_X_scaled = pd.get_dummies(data = train_X_scaled, columns = ['Gender'], prefix = 'Gender')

In [ ]:
train_X_scaled

In [90]:
test_gender_drop = test.drop('Gender', axis=1)

# StandardScaler객체 생성
scaler = StandardScaler()

# StandardScaler 로 데이터 셋 변환 .fit( ) 과 .transform( ) 호출
scaler.fit(test_gender_drop)
test_scaled = scaler.transform(test_gender_drop)

# transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
test_scaled = pd.DataFrame(data=test_scaled, columns=test_gender_drop.columns)

test_scaled['Gender'] = test['Gender']

test_scaled = pd.get_dummies(data = test_scaled, columns = ['Gender'], prefix = 'Gender')

In [ ]:
test_scaled

In [154]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=10, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [155]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=45, mode='min')

In [ ]:
# 모델 학습
model.fit(train_X_scaled, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

## 이상치 임의 제거 

In [273]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [ ]:
train

In [ ]:
sns.pairplot(train)

In [ ]:
# Lenght
var = 'Lenght'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [288]:
train.drop(train[(train['Lenght']> 0.6) & (train['Target'] > 25)].index, inplace=True)
train.drop(train[(train['Lenght']> 0.4) & (train['Target'] > 22.5)].index, inplace=True)
#train.drop(train[(train['Lenght']> 0.4) & (train['Target'] > 20)].index, inplace=True)

In [ ]:
# Diameter
var = 'Diameter'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [ ]:
# Height
var = 'Height'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [ ]:
# Whole Weight
var = 'Whole Weight'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [242]:
train.drop(train[(train['Whole Weight']> 2.5) & (train['Target'] < 12.5)].index, inplace=True)

In [ ]:
# Shucked Weight
var = 'Shucked Weight'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [292]:
train.drop(train[(train['Shucked Weight']> 1.2) & (train['Target'] < 12.5)].index, inplace=True)
#train.drop(train[(train['Shucked Weight']> 1.0) & (train['Target'] < 12.5)].index, inplace=True)
#train.drop(train[(train['Shucked Weight']> 0) & (train['Target'] > 17.5)].index, inplace=True)

In [ ]:
# Viscra Weight	
var = 'Viscra Weight'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [280]:
train.drop(train[(train['Viscra Weight']> 0.6) & (train['Target'] < 12.5)].index, inplace=True)

In [ ]:
# Shell Weight
var = 'Shell Weight'
plt.scatter(x = train[var], y = train['Target'],)
plt.grid(True)

In [284]:
train.drop(train[(train['Shell Weight']> 0.8) & (train['Target'] > 15.0)].index, inplace=True)

In [ ]:
train

In [296]:
train_X = train.drop('Target', axis=1)
train_y = train.Target

In [297]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [307]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=10, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='adam', 
              metrics=['mae'])

In [308]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=45, mode='min')

In [ ]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [305]:
Y_prediction = model.predict(test).flatten()

In [306]:
Y_prediction

array([ 9.477407, 14.252874,  5.288475, ...,  9.170164,  8.542906,
       12.24073 ], dtype=float32)

In [310]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = Y_prediction

submission

submission.to_csv('submission.csv',index=False)

# 피쳐 추가 2

In [199]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [200]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [ ]:
train_X

In [202]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])
train_X['foreign body'] = foreign_body
train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [203]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [204]:
# 껍질의 넓이 ( a * b * π)
area = 0.5 * train_X['Lenght'] * 0.5 * train_X['Diameter'] * np.pi
train_X['Area'] = area

In [205]:
# 껍질의 넓이 
area = 0.5 * test['Lenght'] * 0.5 * test['Diameter'] * np.pi
test['Area'] = area

In [233]:
test['Lenght']**2

0       0.354025
1       0.336400
2       0.067600
3       0.348100
4       0.354025
          ...   
2919    0.028900
2920    0.189225
2921    0.324900
2922    0.211600
2923    0.319225
Name: Lenght, Length: 2924, dtype: float64

In [206]:
# 껍질의 둘레 ( 2π*(0.5 * √(a^2 + b^2)))
perimeter = np.pi * np.sqrt(0.5 * ((train_X['Lenght'] ** 2) + (train_X['Diameter'] ** 2)))
train_X['Perimeter'] = perimeter

In [207]:
# 껍질의 둘레
perimeter = np.pi * np.sqrt(0.5 * ((test['Lenght'] ** 2) + (test['Diameter'] ** 2)))
test['Perimeter'] = perimeter

In [ ]:
train_X

In [ ]:
test

In [ ]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=len(train_X.columns), activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

# 모델 학습
history = model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [226]:
history.history['val_loss'][np.argmin(history.history['val_loss'])]

1.523136019706726

In [227]:
Y_prediction = model.predict(test).flatten()
# 1.5134

In [228]:
Y_prediction

array([ 9.622403 , 14.28157  ,  5.3560038, ...,  8.894803 ,  8.545127 ,
       12.152446 ], dtype=float32)

In [220]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission_1.5134_이물질변경둘레넓이.csv',index=False)

## MinMax scaler

In [216]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [217]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [218]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0


In [219]:
var = ['Lenght','Diameter','Height','Whole Weight','Shucked Weight','Viscra Weight','Shell Weight']

In [220]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(train_X[var])  # 데이터프레임이 되어버림

train_X[var] = scaler.transform(train_X[var])


train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.738806,0.703704,0.386364,0.399062,0.262538,0.381266,0.306387,0,0,1
1,0.477612,0.416667,0.295455,0.133502,0.116123,0.102902,0.101297,0,1,0
2,0.701493,0.740741,0.750000,0.472127,0.355436,0.332454,0.406188,0,1,0
3,0.634328,0.583333,0.659091,0.455529,0.367216,0.428100,0.333333,0,0,1
4,0.298507,0.268519,0.272727,0.042937,0.030629,0.038259,0.036926,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.119403,0.101852,0.045455,0.010824,0.009424,0.005937,0.011976,0,1,0
1249,0.425373,0.407407,0.250000,0.111492,0.101313,0.063984,0.090319,0,1,0
1250,0.619403,0.592593,0.386364,0.276565,0.278357,0.212401,0.176647,1,0,0
1251,0.500000,0.453704,0.363636,0.154249,0.134635,0.141821,0.116267,1,0,0


In [232]:
test[var] = scaler.transform(test[var])

In [241]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])
train_X['foreign body'] = foreign_body
#train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [242]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
#test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [249]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=len(train_X.columns), activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [250]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [ ]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [252]:
Y_prediction = model.predict(test).flatten()

In [253]:
Y_prediction

array([ 9.01175  , 14.216919 ,  5.3847127, ...,  9.154172 ,  8.233047 ,
       11.970468 ], dtype=float32)

In [254]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission.csv',index=False)